In [ ]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Decompose Affine Transformation

$$
A x = B
$$

$$
x = \begin{pmatrix}
a & b & tx\\
c & d & ty\\
0 & 0 & 1
\end{pmatrix}
$$

* http://callumhay.blogspot.co.at/2010/10/decomposing-affine-transforms.html

## Decompose using elements of least-squares fit

In [ ]:
def homogeneous_coords(xy):
    """Return copy of xy with additional dimension (==1).

    Parameters
    ----------
    xy : np.array of shape (n,) or (m, n)
        n dimensions, m values

    Returns
    -------
    np.array of shape (n+1,) or (m, n+1)
        Additional dimension with constant value == 1.
    """
    if xy.ndim == 1:
        newshape = (xy.shape[0]+1, )
        newxy = np.ndarray(shape=newshape)
        newxy[:-1] = xy
        newxy[-1] = 1
        return newxy

    newshape = xy.shape[0], xy.shape[1]+1
    newxy = np.ndarray(shape=newshape)
    newxy[:, :xy.shape[1]] = xy

    newxy[:, -1] = 1
    return newxy

def linear_transformation(a, b):
    """Least squares fit linear transformations of points a to b.

    Parameters
    ----------
    a, b : np.array of shape (n, 2)
        x- and y-coords

    Returns
    -------
    np.array of shape (3, 3)
    """
    xya = homogeneous_coords(a)
    xyb = homogeneous_coords(b)
    # lin. transf. matrix between a and b xy pos = fit
    fit = np.linalg.lstsq(xya, xyb)
    #print fit
    #print "translation=%s" % fit[2,:2]
    return fit[0]

def remove_translation(m):
    """Remove translation from homogeneous transformation matrix."""
    newm = np.zeros(m.shape)
    newm[:2, :2] = m[:2, :2]
    newm[2, 2] = 1
    return newm

def decomp(m):
    """Decompose matrix m into translation, scaling, rotation and shear.

    Used:
    * http://math.stackexchange.com/questions/13150/extracting-rotation-scale-values-from-2d-transformation-matrix?lq=1

    Parameters
    ----------
    m : np.array of shape (3, 3)

    Returns
    -------
    four np.arrays of shape (3, 3)
        Translation, scaling, rotation, and shear matrix. Multiply in that order
        to obtain input matrix: ``np.dot(np.dot(np.dot(msc, mrot), msh), mtr)``.
    """
    tx = m[2, 0]
    ty = m[2, 1]
    a, b, c, d = m[0, 0], m[0, 1], m[1, 0], m[1, 1]
    sx = np.sign(a) * np.sqrt(np.square(a) + np.square(b))
    sy = np.sign(d) * np.sqrt(np.square(c) + np.square(d))
    phi = np.arctan2(-b, a)
    mtr = np.array([[1, 0, 0], [0, 1, 0], [tx, ty, 1]])
    msc = np.array([[sx, 0, 0], [0, sy, 0], [0, 0, 1]])
    mrot = np.array([[np.cos(phi), -np.sin(phi), 0],
                     [np.sin(phi), np.cos(phi), 0],
                     [0, 0, 1]])
    mdecomp = np.dot(np.dot(msc, mrot), mtr)
    msh0 = np.linalg.lstsq(mdecomp[:2, :2], m[:2, :2])
    msh = np.zeros((3, 3))
    msh[:2, :2] += msh0[0]
    msh[2, 2] = 1
    return mtr, msc, mrot, msh

def decomp2(m):
    """Decompose matrix m into translation, scaling, rotation and shear.

    Used:
    * http://math.stackexchange.com/questions/78137/decomposition-of-a-nonsquare-affine-matrix


    Returns
    -------
        mrot.msh.msc.mtr
    """
    c, f = m[2, 0], m[2, 1]
    a, d, b, e = m[0, 0], m[0, 1], m[1, 0], m[1, 1]
    p = np.sqrt(a**2 + b**2)
    r = (a*e - b*d) / p
    q = (a*d + b*e) / (a*e - b*d)
    phi = np.arctan2(-b, a)

    mtr = np.array([[1, 0, 0], [0, 1, 0], [c, f, 1]])
    msc = np.array([[p, 0, 0], [0, r, 0], [0, 0, 1]])
    msh = np.array([[1, q, 0], [1, 0, 0], [0, 0, 1]])
    mrot = np.array([[np.cos(phi), -np.sin(phi), 0],
                     [np.sin(phi), np.cos(phi), 0],
                     [0, 0, 1]])

    raise NotImplementedError("Not implemented correctly!")
    return mtr, msc, msh, mrot

def decomp_scalar(m):
    """
    Returns
    -------
    tuple: tx, ty, sx, sy, phi
        phi in rad
    """
    tx = m[2, 0]
    ty = m[2, 1]
    a, b, c, d = m[0, 0], m[0, 1], m[1, 0], m[1, 1]
    sx = np.sign(a) * np.sqrt(np.square(a) + np.square(b))
    sy = np.sign(d) * np.sqrt(np.square(c) + np.square(d))
    phi = np.arctan2(-b, a)
    return tx, ty, sx, sy, phi

In [ ]:
# design positions
tx, ty = np.meshgrid(
    np.linspace(-2, 2, 2), 
    np.linspace(-2, 2, 2)
)
txy = np.array([tx.flatten(), ty.flatten()]).T
xy = txy
xy += np.array([1, 1])
xy = xy[[0, 1, 3, 2]]
xy

In [ ]:
# distortion matrix
theta = np.pi/8
tx, ty = 0.23, 0.98
sx, sy = 1., 1.
m = np.array(
    [[sx*np.cos(theta), -sx*np.sin(theta), 0], 
     [sy*np.sin(theta), sy*np.cos(theta), 0], 
     [tx, ty, 1]]
)
shxy, shyx = 0., 0.
m = np.array(
    [[sx*np.cos(theta) + sx*np.sin(theta)*shxy, -sx*np.sin(theta), 0], 
     [sy*np.sin(theta), sy*np.cos(theta), 0], 
     [tx, ty, 1]]
)
m = np.array(
    [[0.95, 0.86, 0],
     [0.07, 1.13, 0],
     [1.5, -0.52, 1]]
)

m

In [ ]:
mtr, msc, mrot, msh = decomp(m)
tx, ty, sx, sy, phi = decomp_scalar(m)
phi / pi * 180

In [ ]:
mall = np.dot(np.dot(msc, np.dot(mrot, msh)), mtr)
mall, (np.abs(mall - m) < 1e-15).all()
#msh
#np.dot(msc, np.dot(mrot, mtr))

In [ ]:
# measured positions
#m = np.dot(msc, mrot)
xym = np.dot(homogeneous_coords(xy), m)
xym

In [ ]:
# fit of design positions to measured positions
mfit, _, _, _ = np.linalg.lstsq(homogeneous_coords(xy), xym)
mfit

In [ ]:
# fitted positions
xyf = np.dot(homogeneous_coords(xy), mfit)
xyf

In [ ]:
mfittr, mfitsc, mfitrot, mfitsh = decomp(mfit)
mfitall = np.dot(np.dot(np.dot(mfitsc, mfitrot), mfitsh), mfittr)
mfitall, (np.abs(mfitall - m) < 1e-15).all()

In [ ]:
mfitall = np.dot(np.dot(np.dot(mfittr, mfitrot), mfitsc), mfitsh)
mfitall, (np.abs(mfitall - m) < 1e-15).all()

In [ ]:
xyf1 = np.dot(homogeneous_coords(xy), 
              np.dot(np.dot(mfitsc, np.dot(mfitrot, mfitsh)), mfittr))
#xyf1 = np.dot(homogeneous_coords(xy), mfit)

In [ ]:
# deviations
xyc = xym[:, :2] - xy
xyc

In [ ]:
# plot design positions, measured positions, and deviations
fig2 = plt.figure()
fig2.set_size_inches(7.5, 7.5)

ax20 = fig2.add_subplot(1, 1, 1)
ax20.add_patch(Polygon(xy, closed=True, fill=True, alpha=0.3, color="k"))
ax20.scatter(xy[:, 0], xy[:, 1], color="k")
ax20.scatter(xym[:, 0], xym[:, 1], color="r")
#ax20.scatter(xyf1[:, 0], xyf1[:, 1], color="g")
ax20.quiver(xy[:,0], xy[:,1], xyc[:, 0], xyc[:, 1],
            units="xy", scale=1e0)


mtmp0 = np.dot(np.dot(np.dot(mfitsc, mfitrot), mfitsh), mfittr)
#mtmp0 = np.dot(np.dot(np.dot(mfittr, mfitsc), mfitrot), mfitsh)
xytmp0 = np.dot(homogeneous_coords(xy), mtmp0)
ax20.add_patch(Polygon(xytmp0[:, :2], closed=True, fill=True, alpha=0.3, color="pink"))
ax20.scatter(xytmp0[:, 0], xytmp0[:, 1], 
             color="pink")

#mtmp1 = np.dot(np.dot(mfitsc, mfitrot), mfittr)
mtmp1 = np.dot(np.dot(mfitrot, mfitsh), mfittr)
xytmp1 = np.dot(homogeneous_coords(xy), mtmp1)
ax20.add_patch(Polygon(xytmp1[:, :2], closed=True, fill=True, alpha=0.3, color="orange"))
ax20.scatter(xytmp1[:, 0], xytmp1[:, 1],
             color="orange")

mtmp2 = np.dot(mfitrot, mfittr)
xytmp2 = np.dot(homogeneous_coords(xy), mtmp2)
ax20.add_patch(Polygon(xytmp2[:, :2], closed=True, fill=True, alpha=0.3, color="y"))
ax20.scatter(xytmp2[:, 0], xytmp2[:, 1], 
             color="y")

mtmp3 = mfittr
xytmp3 = np.dot(homogeneous_coords(xy), mtmp3)
ax20.add_patch(Polygon(xytmp3[:, :2], closed=True, fill=True, alpha=0.3, color="blue"))
ax20.scatter(xytmp3[:, 0], xytmp3[:, 1], 
             color="blue")

ax20.set_xlim(-6, 6)
ax20.set_ylim(-6, 6)
ax20.grid()

**Least squares fit accuracy**

In [ ]:
d = xyf[:, :2] - xym[:, :2]
d, d.mean(axis=0), d.std(axis=0)

## Decompose into translation and rotation with Kabsch Algorithm

In [ ]:
def kabsch_rmsd(xy1, xy2):
    """Calculate the optimal rotation that minimizes the RMSD between 
    two sets of points.
    
    * http://en.wikipedia.org/wiki/Kabsch_algorithm
    """
    # Center xy1 and xy2.
    xy1_c = xy1.mean(axis=0)
    xy2_c = xy2.mean(axis=0)
    #print "removing centers", xy1_c, xy2_c
    a = xy1 - xy1_c
    b = xy2 - xy2_c
    
    # Covariance matrix.
    c = np.dot(a.T, b)
    #print "covariance matrix", c
    
    # Calculate optimal rotation matrix (I).
    v, s, w = np.linalg.svd(c)
    
    # Ensure right-handed coordinate system.
    if (np.linalg.det(v) * np.linalg.det(w)) < 0.0:
        s[-1] = -s[-1]
        v[:, -1] = -v[:, -1]

    # Calculate optimal rotation matrix (II).
    #print "v, w", v, w
    m = np.dot(v, w)
    
    return m

In [ ]:
krot = kabsch_rmsd(xy, xym[:, :2])
kphi= np.arctan2(-krot[0, 1], krot[0, 0])
mphi= np.arctan2(-mfitrot[0, 1], mfitrot[0, 0])

krot, kphi / pi * 180, mfitrot, mphi / pi * 180

In [ ]:
# plot design positions, measured positions, and deviations
fig2 = plt.figure()
fig2.set_size_inches(7.5, 7.5)

ax20 = fig2.add_subplot(1, 1, 1)
ax20.add_patch(Polygon(xy, closed=True, fill=True, alpha=0.3, color="k"))
ax20.scatter(xy[:, 0], xy[:, 1], color="k")
ax20.add_patch(Polygon(xym[:, :2], closed=True, fill=True, alpha=0.3, color="r"))
ax20.scatter(xym[:, 0], xym[:, 1], color="r")
#ax20.quiver(xy[:,0], xy[:,1], xyc[:, 0], xyc[:, 1],
#            units="xy", scale=1e0)

xy_c = xy.mean(axis=0)
xym_c = xym[:, :2].mean(axis=0)
xytmp = np.dot(xy - xy_c, krot) + xym_c
ax20.add_patch(Polygon(xytmp, closed=True, fill=True, alpha=0.3, color="y"))
ax20.scatter(xytmp[:, 0], xytmp[:, 1],
             color="y")

xytmp2 = xym #np.dot(homogeneous_coords(xy), m)
ax20.add_patch(Polygon(xytmp2[:, :2], closed=True, fill=True, alpha=0.3, color="pink"))
ax20.scatter(xytmp2[:, 0], xytmp2[:, 1], 
             color="pink")

ax20.set_xlim(-6, 6)
ax20.set_ylim(-6, 6)
ax20.grid()

## Decompose fitting each component separately

### 1. Fit Translation

In [ ]:
# fit translation (subtract means)
#xy, xym[:, :2]
a = xy - xy.mean(axis=0)
b = xym[:, :2] - xym[:, :2].mean(axis=0)

tr = xym[:, :2].mean(axis=0) - xy.mean(axis=0)
tr, mfittr

In [ ]:
# plot design positions, measured positions, and deviations
fig2 = plt.figure()
fig2.set_size_inches(7.5, 7.5)

ax20 = fig2.add_subplot(1, 1, 1)

xytmp0 = a
ax20.add_patch(Polygon(xytmp0, closed=True, fill=True, alpha=0.3, color="k"))
ax20.scatter(xytmp0[:, 0], xytmp0[:, 1], color="k")

xytmp1 = a # + xy.mean(axis=0) + tr
ax20.add_patch(Polygon(xytmp1, closed=True, fill=True, alpha=0.3, color="blue"))
ax20.scatter(xytmp1[:, 0], xytmp1[:, 1], color="blue")

xytmp2 = b #+ xym[:, :2].mean(axis=0)
ax20.add_patch(Polygon(xytmp2[:, :2], closed=True, fill=True, alpha=0.3, color="pink"))
ax20.scatter(xytmp2[:, 0], xytmp2[:, 1], color="pink")

ax20.set_xlim(-6, 6)
ax20.set_ylim(-6, 6)
ax20.grid()

### 2. Fit Rotation

In [ ]:
import scipy.optimize

In [ ]:
def to_mrot(phi):
    return np.array(
        [[np.cos(phi), -np.sin(phi)],
         [np.sin(phi), np.cos(phi)]]
    )

In [ ]:
# fit rotation (after translation)
a = xy - xy.mean(axis=0)
xytmp = xym #np.dot(xy - xy_c, krot) + xym_c
b = xytmp[:, :2] - xytmp[:, :2].mean(axis=0)
#a, b
funx = lambda phi: np.sum(np.cos(phi) * a[:, 0] - np.sin(phi) * a[:, 1] - b[:, 0])
funy = lambda phi: np.sum(np.sin(phi) * a[:, 0] + np.cos(phi) * a[:, 1] - b[:, 1])
fun = lambda phi: np.sum((np.cos(phi) * a[:, 0] - np.sin(phi) * a[:, 1] - b[:, 0])**2 + 
                         (np.sin(phi) * a[:, 0] + np.cos(phi) * a[:, 1] - b[:, 1])**2)
funxphi = scipy.optimize.brent(funx)
funyphi = scipy.optimize.brent(funy)
funphi1 = scipy.optimize.brent(fun)
funphi2 = scipy.optimize.fmin(fun, np.array([0]))[0]
funphi3 = scipy.optimize.minimize_scalar(fun)["x"]
funphi = funphi1
print funxphi, funyphi, funphi1, funphi2, funphi3
to_mrot(-funphi)

In [ ]:
fig2 = plt.figure()
fig2.set_size_inches(7.5, 7.5)

ax20 = fig2.add_subplot(1, 1, 1)

xytmp0 = a
ax20.add_patch(Polygon(xytmp0, closed=True, fill=True, alpha=0.3, color="k"))
ax20.scatter(xytmp0[:, 0], xytmp0[:, 1], color="k")

#xytmp1 = np.dot(a, mrot[:2, :2])
xytmp1 = np.dot(a, to_mrot(-funphi))
#xytmp1 = np.dot(a, kabsch_rmsd(a, b))
ax20.add_patch(Polygon(xytmp1, closed=True, fill=True, alpha=0.3, color="y"))
ax20.scatter(xytmp1[:, 0], xytmp1[:, 1], color="y")

xytmp2 = b
ax20.add_patch(Polygon(xytmp2[:, :2], closed=True, fill=True, alpha=0.3, color="pink"))
ax20.scatter(xytmp2[:, 0], xytmp2[:, 1], 
             color="pink")

ax20.set_xlim(-6, 6)
ax20.set_ylim(-6, 6)
ax20.grid()

### 3. Fit Scaling

In [ ]:
def to_msc(s):
    return np.array(
        [[s[0], 0],
         [0, s[1]]]
        )

In [ ]:
# fit scaling (after translation and rotation)
a = xy - xy.mean(axis=0)
xytmp = xym
b = np.dot((xytmp[:, :2] - xytmp[:, :2].mean(axis=0)), to_mrot(funphi))
#a, b
fun = lambda s: np.sum((s[0]*a[:, 0] - b[:, 0])**2 + 
                       (s[1]*a[:, 1] - b[:, 1])**2)
funsc = scipy.optimize.minimize(fun, (1, 1))["x"]
to_msc(funsc)

In [ ]:
fig2 = plt.figure()
fig2.set_size_inches(7.5, 7.5)

ax20 = fig2.add_subplot(1, 1, 1)

xytmp0 = a
ax20.add_patch(Polygon(xytmp0, closed=True, fill=True, alpha=0.3, color="k"))
ax20.scatter(xytmp0[:, 0], xytmp0[:, 1], color="k")

#xytmp1 = np.dot(a, mrot[:2, :2])
xytmp1 = np.dot(a, to_msc(funsc))
#xytmp1 = np.dot(a, kabsch_rmsd(a, b))
ax20.add_patch(Polygon(xytmp1, closed=True, fill=True, alpha=0.3, color="red"))
ax20.scatter(xytmp1[:, 0], xytmp1[:, 1], color="red")

xytmp2 = b
#xytmp2 = np.dot(a, to_mrot(-funphi))
#xytmp2 = np.dot(a, kabsch_rmsd(a, b))
ax20.add_patch(Polygon(xytmp2[:, :2], closed=True, fill=True, alpha=0.3, color="pink"))
ax20.scatter(xytmp2[:, 0], xytmp2[:, 1], 
             color="pink")

ax20.set_xlim(-6, 6)
ax20.set_ylim(-6, 6)
ax20.grid()

### 4. Fit shear / orthogonality error

In [ ]:
def to_msh(s):
    return np.array(
        [[1, s[0]],
         [s[1], 1]]
        )

In [ ]:
# fit shear (after translation, rotation and scaling)
a = xy - xy.mean(axis=0)
xytmp = xym
b = np.dot(np.dot((xytmp[:, :2] - xytmp[:, :2].mean(axis=0)), to_mrot(funphi)), 
           np.linalg.inv(to_msc(funsc)))
#a, b
fun = lambda s: np.sum((a[:, 0] + s[0]*a[:, 1] - b[:, 0])**2 + 
                       (s[1]*a[:, 0] + a[:, 1] - b[:, 1])**2)
funsh = scipy.optimize.minimize(fun, (0, 0))["x"]
to_msh(funsh)

In [ ]:
fig2 = plt.figure()
fig2.set_size_inches(7.5, 7.5)

ax20 = fig2.add_subplot(1, 1, 1)

xytmp0 = a
ax20.add_patch(Polygon(xytmp0, closed=True, fill=True, alpha=0.3, color="k"))
ax20.scatter(xytmp0[:, 0], xytmp0[:, 1], color="k")

xytmp1 = np.dot(a, to_msh(funsh))
ax20.add_patch(Polygon(xytmp1, closed=True, fill=True, alpha=0.3, color="cyan"))
ax20.scatter(xytmp1[:, 0], xytmp1[:, 1], color="cyan")

xytmp2 = b
ax20.add_patch(Polygon(xytmp2[:, :2], closed=True, fill=True, alpha=0.3, color="pink"))
ax20.scatter(xytmp2[:, 0], xytmp2[:, 1], color="pink")

ax20.set_xlim(-6, 6)
ax20.set_ylim(-6, 6)
ax20.grid()

### 5. Recompose shear, scale, rotation and translation

In [ ]:
# center
a0 = (xy - xy.mean(axis=0))
# shear
xy2 = np.dot(a0, to_msh(funsh)) 
# scale
xy3 = np.dot(np.dot(a0, to_msh(funsh)), to_msc(funsc))
# rotate
xy4 = np.dot(np.dot(np.dot(a0, to_msh(funsh)), to_msc(funsc)), to_mrot(-funphi))
# translate
xy5 = xy4 + xym[:, :2].mean(axis=0)

In [ ]:
fig2 = plt.figure()
fig2.set_size_inches(7.5, 7.5)
ax20 = fig2.add_subplot(1, 1, 1)


ax20.add_patch(Polygon(xy, closed=True, fill=True, alpha=0.3, color="k"))
ax20.scatter(xy[:, 0], xy[:, 1], color="k")
ax20.add_patch(Polygon(xym[:, :2], closed=True, fill=True, alpha=0.3, color="green"))
ax20.scatter(xym[:, 0], xym[:, 1], color="green")
#ax20.quiver(xy[:,0], xy[:,1], xyc[:, 0], xyc[:, 1], units="xy", scale=1e0)

xytmp1 = a0
ax20.add_patch(Polygon(xytmp1[:, :2], closed=True, fill=True, alpha=0.3, color="gray"))
ax20.scatter(xytmp1[:, 0], xytmp1[:, 1], color="gray")

xytmp2 = xy2
ax20.add_patch(Polygon(xytmp2[:, :2], closed=True, fill=True, alpha=0.3, color="cyan"))
ax20.scatter(xytmp2[:, 0], xytmp2[:, 1], color="cyan")

xytmp3 = xy3
ax20.add_patch(Polygon(xytmp3[:, :2], closed=True, fill=True, alpha=0.3, color="red"))
ax20.scatter(xytmp3[:, 0], xytmp3[:, 1], color="red")

xytmp4 = xy4
ax20.add_patch(Polygon(xytmp4[:, :2], closed=True, fill=True, alpha=0.3, color="pink"))
ax20.scatter(xytmp4[:, 0], xytmp4[:, 1], color="pink")

xytmp5 = xy5
ax20.add_patch(Polygon(xytmp5[:, :2], closed=True, fill=True, alpha=0.3, color="blue"))
ax20.scatter(xytmp5[:, 0], xytmp5[:, 1], color="blue")


ax20.set_xlim(-6, 6)
ax20.set_ylim(-6, 6)
ax20.grid()

**Separate component fit accuracy**

In [ ]:
d = xy5 - xym[:, :2]
d, d.mean(axis=0), d.std(axis=0)

### 6. Fit linear transformation after separate component fit

In [ ]:
a = xy5 - xy5.mean(axis=0)
b = xym[:, :2] - xym[:, :2].mean(axis=0)

# least squares fit
m = np.linalg.lstsq(a, b)[0]

# fit rotation
#fun = lambda phi: np.sum((np.cos(phi) * a[:, 0] - np.sin(phi) * a[:, 1] - b[:, 0])**2 + 
#                         (np.sin(phi) * a[:, 0] + np.cos(phi) * a[:, 1] - b[:, 1])**2)
#funphi = scipy.optimize.brent(fun)
#m = to_mrot(-funphi)

# don't fit
#m = np.eye(2)
#m

In [ ]:
# plot design positions, measured positions, and deviations
fig2 = plt.figure()
fig2.set_size_inches(7.5, 7.5)

ax20 = fig2.add_subplot(1, 1, 1)

xytmp0 = a
ax20.add_patch(Polygon(xytmp0, closed=True, fill=True, alpha=0.3, color="k"))
ax20.scatter(xytmp0[:, 0], xytmp0[:, 1], color="k")

xytmp1 = np.dot(a, m)
ax20.add_patch(Polygon(xytmp1, closed=True, fill=True, alpha=0.3, color="cyan"))
ax20.scatter(xytmp1[:, 0], xytmp1[:, 1], color="cyan")

xytmp1 = b
ax20.add_patch(Polygon(xytmp1, closed=True, fill=True, alpha=0.3, color="pink"))
ax20.scatter(xytmp1[:, 0], xytmp1[:, 1], color="pink")

ax20.set_xlim(-6, 6)
ax20.set_ylim(-6, 6)
ax20.grid()

**Separate component fit plus linear transformation least squares fit accuracy**

In [ ]:
d = np.dot(a, m) - b
d, d.mean(axis=0), d.std(axis=0)

## Write to DataFrame and csv-file

In [ ]:
df = pd.DataFrame(dtype=np.float64)
df["x pos design"] = xy[:,0]
df["y pos design"] = xy[:,1]
df["x dev"] = xyc[:,0]
df["y dev"] = xyc[:,1]
df.info()

In [ ]:
print df.to_csv(sep=";", float_format="%12.5f")

In [ ]:
#df["y dev"][5] = np.nan
#df["x dev"][4] = np.nan
#print df.to_csv(sep=";", float_format="%12.5f", na_rep=" "*12)